In [2]:
import numpy as np

In [3]:
state = "000000000"
state_list = []
state_list.append(state)
trans_matrix = {}

In [4]:
def create_trans_matrix(trans_matrix, state_list, state):
    for state in state_list:
        trans_matrix[state] = {}
        for index, pos in enumerate(state):
            if(pos == '0'):
                intermediate_state = state[:index] + '1' + state[index + 1:]
                trans_matrix[state][index] = {}

                sum = 0
                for index2, pos2 in enumerate(intermediate_state):
                    if(pos2 == '0'):
                        new_state = intermediate_state[:index2] + '2' + intermediate_state[index2 + 1:]
                        if(new_state not in state_list):
                            state_list.append(new_state)
                        trans_matrix[state][index][new_state] = np.random.random()
                        sum += trans_matrix[state][index][new_state]
                trans_matrix[state][index] = { k:v/sum for (k,v) in trans_matrix[state][index].items()}

In [5]:
def pretty(d, indent=0):
   for key, value in d.items():
      print('\t' * indent + str(key), end=' ')
      if isinstance(value, dict):
         pretty(value, indent+1)
      else:
         print('\t' * (indent+1) + str(value))

In [6]:
create_trans_matrix(trans_matrix, state_list, state)
# print(state_list)
# print(len(state_list))

# print(trans_matrix)
print(len(trans_matrix))
pretty(trans_matrix['120000000'])

3139
2 	121200000 		0.051284254325824764
	121020000 		0.23454867466970927
	121002000 		0.10749138601065966
	121000200 		0.21245239252209136
	121000020 		0.13412023829822534
	121000002 		0.26010305417348956
3 	122100000 		0.12154779652043987
	120120000 		0.35265401974773064
	120102000 		0.08348999209380427
	120100200 		0.03866017324991036
	120100020 		0.15674874158985172
	120100002 		0.24689927679826318
4 	122010000 		0.183229464776322
	120210000 		0.01625339768742129
	120012000 		0.1801075608003543
	120010200 		0.10257707556626817
	120010020 		0.2745521843861069
	120010002 		0.2432803167835272
5 	122001000 		0.06146617104113444
	120201000 		0.17945580372350206
	120021000 		0.04707864467121933
	120001200 		0.17249839557437613
	120001020 		0.23779545683840703
	120001002 		0.30170552815136104
6 	122000100 		0.45784202349371994
	120200100 		0.02972268797195235
	120020100 		0.09822841757922174
	120002100 		0.20951603269068522
	120000120 		0.03834150698774582
	120000102 		0.1663493312766749


In [7]:
def board_to_state(board):
    state = "000000000"
    for x, row in enumerate(board):
        for y, cell in enumerate(row):
            if cell == 1:
                state = state[:y + 3*x] + '1' + state[y + 3*x + 1:]
            elif cell == -1:
                state = state[:y + 3*x] + '2' + state[y + 3*x + 1:]

    return state
                
def state_to_board(state):
    board = []
    for x in range (3):
        row = [0] * 3
        board.append(row)
    
    for index, value in enumerate(state):
        if value == '1':
            board[int(index)//3][int(index)%3] = 1
        elif value == '2':
            board[int(index)//3][int(index)%3] = -1
    
    return board

In [14]:
state = "222000011"
# print(state == board_to_state(state_to_board(state)))
state.count('0')

4

In [15]:
import json
from random import randrange

def create_policy(trans_matrix):
    policy = {}
    for state in trans_matrix:
        selected_index = randrange(state.count('0'))
        c = 0
        for index, pos in enumerate(state):
            if pos == '0':
                if c == selected_index:
                    policy[state] = index
                    break
                c = c + 1

    with open('policy.json', 'w') as f:
        json.dump(policy, f)

create_policy(trans_matrix)

In [16]:
with open('policy.json') as f: 
    data = f.read() 
policy = json.loads(data)
len(policy)

3139

In [10]:
#import modules
import pygame
from pygame.locals import *
import numpy as np

pygame.init()

screen_height = 300
screen_width = 300
line_width = 6
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption('Tic Tac Toe')

#define colours
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)

#define font
font = pygame.font.SysFont(None, 40)

#define variables
clicked = False
player = 1
pos = (0,0)
markers = []
game_over = False
winner = 0

#setup a rectangle for "Play Again" Option
again_rect = Rect(screen_width // 2 - 80, screen_height // 2, 160, 50)

#create empty 3 x 3 list to represent the grid
for x in range (3):
	row = [0] * 3
	markers.append(row)



def draw_board():
	bg = (255, 255, 210)
	grid = (50, 50, 50)
	screen.fill(bg)
	for x in range(1,3):
		pygame.draw.line(screen, grid, (0, 100 * x), (screen_width,100 * x), line_width)
		pygame.draw.line(screen, grid, (100 * x, 0), (100 * x, screen_height), line_width)

def draw_markers():
	x_pos = 0
	for x in markers:
		y_pos = 0
		for y in x:
			if y == 1:
				pygame.draw.line(screen, red, (x_pos * 100 + 15, y_pos * 100 + 15), (x_pos * 100 + 85, y_pos * 100 + 85), line_width)
				pygame.draw.line(screen, red, (x_pos * 100 + 85, y_pos * 100 + 15), (x_pos * 100 + 15, y_pos * 100 + 85), line_width)
			if y == -1:
				pygame.draw.circle(screen, green, (x_pos * 100 + 50, y_pos * 100 + 50), 38, line_width)
			y_pos += 1
		x_pos += 1	


def check_game_over():
	global game_over
	global winner

	x_pos = 0
	for x in markers:
		#check columns
		if sum(x) == 3:
			winner = 1
			game_over = True
		if sum(x) == -3:
			winner = 2
			game_over = True
		#check rows
		if markers[0][x_pos] + markers [1][x_pos] + markers [2][x_pos] == 3:
			winner = 1
			game_over = True
		if markers[0][x_pos] + markers [1][x_pos] + markers [2][x_pos] == -3:
			winner = 2
			game_over = True
		x_pos += 1

	#check cross
	if markers[0][0] + markers[1][1] + markers [2][2] == 3 or markers[2][0] + markers[1][1] + markers [0][2] == 3:
		winner = 1
		game_over = True
	if markers[0][0] + markers[1][1] + markers [2][2] == -3 or markers[2][0] + markers[1][1] + markers [0][2] == -3:
		winner = 2
		game_over = True

	#check for tie
	if game_over == False:
		tie = True
		for row in markers:
			for i in row:
				if i == 0:
					tie = False
		#if it is a tie, then call game over and set winner to 0 (no one)
		if tie == True:
			game_over = True
			winner = 0

def draw_game_over(winner):

	if winner != 0:
		end_text = "Player " + str(winner) + " wins!"
	elif winner == 0:
		end_text = "You have tied!"

	end_img = font.render(end_text, True, blue)
	pygame.draw.rect(screen, green, (screen_width // 2 - 100, screen_height // 2 - 60, 200, 50))
	screen.blit(end_img, (screen_width // 2 - 100, screen_height // 2 - 50))

	again_text = 'Play Again?'
	again_img = font.render(again_text, True, blue)
	pygame.draw.rect(screen, green, again_rect)
	screen.blit(again_img, (screen_width // 2 - 80, screen_height // 2 + 10))

    
#*********************************Custom Functions************************************
def create_trans_matrix(trans_matrix):
    state_list = []
    state_list.append("000000000")
    
    for state in state_list:
        trans_matrix[state] = {}
        for index, pos in enumerate(state):
            if(pos == '0'):
                intermediate_state = state[:index] + '1' + state[index + 1:]
                trans_matrix[state][index] = {}

                sum = 0
                for index2, pos2 in enumerate(intermediate_state):
                    if(pos2 == '0'):
                        new_state = intermediate_state[:index2] + '2' + intermediate_state[index2 + 1:]
                        if(new_state not in state_list):
                            state_list.append(new_state)
                        trans_matrix[state][index][new_state] = np.random.random()
                        sum += trans_matrix[state][index][new_state]
                trans_matrix[state][index] = { k:v/sum for (k,v) in trans_matrix[state][index].items()}

def board_to_state(board):
    state = "000000000"
    for x, row in enumerate(board):
        for y, cell in enumerate(row):
            if cell == 1:
                state = state[:y + 3*x] + '1' + state[y + 3*x + 1:]
            elif cell == -1:
                state = state[:y + 3*x] + '2' + state[y + 3*x + 1:]

    return state
                
def state_to_board(state):
    board = []
    for x in range (3):
        row = [0] * 3
        board.append(row)
    
    for index, value in enumerate(state):
        if value == '1':
            board[int(index)//3][int(index)%3] = 1
        elif value == '2':
            board[int(index)//3][int(index)%3] = -1
    
    return board
#************************************************************************************


# Create Probability Transition matrix.
trans_matrix = {}
create_trans_matrix(trans_matrix)

#main loop
run = True
while run:

    #draw board and markers first
    draw_board()
    draw_markers()

    #handle events
    for event in pygame.event.get():
        #handle game exit
        if event.type == pygame.QUIT:
            run = False
        #run new game
        if game_over == False:
#             #check for mouseclick
#             if event.type == pygame.MOUSEBUTTONDOWN and clicked == False:
#                 clicked = True
#             if event.type == pygame.MOUSEBUTTONUP and clicked == True:
#                 clicked = False
#                 pos = pygame.mouse.get_pos()
#                 cell_x = pos[0] // 100
#                 cell_y = pos[1] // 100
#                 if markers[cell_x][cell_y] == 0:
            initial_state = board_to_state(markers)
            position = cell_y + 3*cell_x
            next_possible_states = trans_matrix[initial_state][position]
            if next_possible_states:	# if next_possible_states are empty. i.e current state is not 4X, 4O state.
                next_state = max(next_possible_states, key=next_possible_states.get) # Select most probable state

            # Player's move
            cell_x = policy[initial_state] / 3
            cell_y = policy[initial_state] % 3
            markers[cell_x][cell_y] = player
            check_game_over()

            if game_over == False:
                # Computer's move
                markers = state_to_board(next_state)
                check_game_over()


    #check if game has been won
    if game_over == True:
        draw_game_over(winner)
        #check for mouseclick to see if we clicked on Play Again
        if event.type == pygame.MOUSEBUTTONDOWN and clicked == False:
            clicked = True
        if event.type == pygame.MOUSEBUTTONUP and clicked == True:
            clicked = False
            pos = pygame.mouse.get_pos()
            if again_rect.collidepoint(pos):
                #reset variables
                game_over = False
                player = 1
                pos = (0,0)
                markers = []
                winner = 0
                #create empty 3 x 3 list to represent the grid
                for x in range (3):
                    row = [0] * 3
                    markers.append(row)
                
				# These two lines create new transition matrix for every run.
                trans_matrix = {}
                create_trans_matrix(trans_matrix)


    #update display
    pygame.display.update()

pygame.quit()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


NameError: name 'cell_y' is not defined